In [1]:
import sys
p="/home/kpalin/software/huddinge_tsne_browser"
if p not in sys.path:
    sys.path.append(p)
import numpy as np
import holoviews as hv
import numpy as np
import pandas as pd
hv.extension('bokeh',logo=False)

In [2]:
import huddinge_tsne_browser.tsne_mapper as htm 

reload(htm)


<module 'huddinge_tsne_browser.tsne_mapper' from '/home/kpalin/software/huddinge_tsne_browser/huddinge_tsne_browser/tsne_mapper.pyc'>

In [4]:
%%bash
K=8
cd ~/software/MODER/data
for i in *.fastq.gz
do
    OUT=$(basename $i .fastq.gz).${K}mer_counts.jf
    echo $OUT
    zcat $i | /usr/bin/time -v jellyfish count -o $OUT --text -m ${K} -s 1M --bf-size 1G -t 16 --disk /dev/stdin
done

HNF4A_TGACAG20NGA_AF_1.8mer_counts.jf
HNF4A_TGACAG20NGA_AF_2.8mer_counts.jf
HNF4A_TGACAG20NGA_AF_3.8mer_counts.jf
HNF4A_TGACAG20NGA_AF_4.8mer_counts.jf


	Command being timed: "jellyfish count -o HNF4A_TGACAG20NGA_AF_1.8mer_counts.jf --text -m 8 -s 1M --bf-size 1G -t 16 --disk /dev/stdin"
	User time (seconds): 2.31
	System time (seconds): 2.54
	Percent of CPU this job got: 413%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:01.17
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 1238588
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 185383
	Voluntary context switches: 5920
	Involuntary context switches: 681
	Swaps: 0
	File system inputs: 0
	File system outputs: 4656
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0
	Command being timed: "jellyfish count -o HNF4A_TGACAG20NGA_AF_2.8mer_counts.jf --text -m 8 -s 1M --bf-size 1G -t 16 --disk /dev/stdin"
	User time (seconds): 2.5

In [3]:
tsne = htm.TsneMapper("/home/kpalin/software/huddinge_tsne_browser/kmer8_iters4k.tsne")
kmer_size=8
for i in range(4):
    tsne.add_kmercounts("HNF4A_{}".format(i+1),
                        "/home/kpalin/software/MODER/data/HNF4A_TGACAG20NGA_AF_{}.{}mer_counts.jf".format(i+1,kmer_size))
#tsne.add_kmercounts("HNF4A_4","/home/kpalin/software/MODER/data/HNF4A_TGACAG20NGA_AF_4.mer_counts.jf")

INFO:root:Read 65536 sequences.
INFO:root:Setting embedding from input data


In [4]:
if False:
    tsne = htm.TsneMapper("/home/kpalin/software/huddinge_tsne_browser/HNF4A_top1k2.tsne")
    tsne.add_kmercounts("HNF4A_1","/home/kpalin/software/MODER/data/HNF4A_TGACAG20NGA_AF_1.mer_counts.jf")
    tsne.add_kmercounts("HNF4A_4","/home/kpalin/software/MODER/data/HNF4A_TGACAG20NGA_AF_4.mer_counts.jf")

In [5]:
import huddinge_tsne_browser.huddinge_browser as hhb
import huddinge_tsne_browser.datashaderselect
reload(huddinge_tsne_browser.datashaderselect)
reload(hhb)


<module 'huddinge_tsne_browser.huddinge_browser' from '/home/kpalin/software/huddinge_tsne_browser/huddinge_tsne_browser/huddinge_browser.pyc'>

In [6]:
x=hhb.HuddingBrowser(tsne)
x.holoview_plot()

INFO:root:Initialized DataShaderSelect


datashade Value <class 'datashader.reductions.mean'> :HoloMap   [Counts]
   :Points   [tsne0,tsne1]   (Sequence,Value)
[]


:Layout
   .DynamicMap.I   :DynamicMap   [Counts]
   .DynamicMap.II  :DynamicMap   []
   .DynamicMap.III :DynamicMap   []

In [56]:
from collections import Counter
pd.DataFrame(map(Counter,zip(*x.selected.index))).fillna(0)

,A,C,G,T
0,10,29,2.0,2.0
1,2,40,1.0,0.0
2,7,23,12.0,1.0
3,27,3,12.0,1.0
4,41,1,1.0,0.0
5,25,17,0.0,1.0
6,9,19,0.0,15.0
7,6,28,3.0,6.0
8,3,37,1.0,2.0
9,17,21,1.0,4.0


In [47]:
print "\n".join(x.selected.index)

AGGACTTTGA
GGACCTTGGA
GGACTTTGAC
GTGGACCTTG
GGACTTTGCC
CGACCTTTGA
GAACTTTGAC
GCACTTTGAC
GGACCTTGAA
GAACTTTGAA
TAGACTTTGA
GGACGTTGAC
TGGACGTTGA
ATGAACCTTG
GGCCTTTGAC
GACTTTGAGC
TGAACCTTGC
ACGAACTTTG
GGACTTTGAG
CGCACTTTGA
TGCACTTTGA
TGTACTTTGT
TGTACTTCGA
GAACCTTGAC
ACGGACCTTG
GCAAACTTTG
CGAACTTTGA
TGTACTTTGA
AGACTTTGAA
GAACTTTGCA
TAGACATTTA
TGCCCTTTGA
CGGACCTTGC
GACTTTGATA
AGAACTTTGA
CCGAACTTTG
TTTACTGAGA
CTCGAACTTA
TCGAACTTTG
